In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import time
import re
import tensorflow as tf
import tensorflow_hub as hub
import pickle
import os
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import string
%matplotlib inline

os.chdir("/Users/alexdessouky/Desktop/MIDS/w266")

#load training data
twitter_train = pd.read_excel('./w266_final_project/StanceDataset/train.xlsx')

#load test data
twitter_test = pd.read_excel('./w266_final_project/StanceDataset/test.xlsx')

In [4]:
twitter_train.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
0,"@tedcruz And, #HandOverTheServer she wiped cle...",Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
1,Hillary is our best choice if we truly want to...,Hillary Clinton,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
2,@TheView I think our country is ready for a fe...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
3,I just gave an unhealthy amount of my hard-ear...,Hillary Clinton,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,@PortiaABoulger Thank you for adding me to you...,Hillary Clinton,NONE,3. The tweet is not explicitly expressing opi...,pos


In [5]:
twitter_test.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
0,He who exalts himself shall be humbled; a...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,pos
1,RT @prayerbullets: I remove Nehushtan -previou...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,other
2,@Brainman365 @heidtjj @BenjaminLives I have so...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,pos
3,#God is utterly powerless without Human interv...,Atheism,AGAINST,1. The tweet explicitly expresses opinion abo...,neg
4,@David_Cameron Miracles of #Multiculturalism...,Atheism,AGAINST,2. The tweet does NOT expresses opinion about ...,neg


## Code for loading full Elmo outputs from pickle files and prepping labelled data

In [6]:
# ELMO OUTPUTS
with open("./ELMO Outputs v.2/elmo_train.pickle", "rb") as f:
    train_vectors = pickle.load(f)
    
with open("./ELMO Outputs v.2/elmo_test.pickle", "rb") as f:
    test_vectors = pickle.load(f)

In [10]:
#PREP LABELS
stance_labels_train = np.array(twitter_train['Stance'].apply(lambda x: 
                                                                    2 if x == "FAVOR" else 
                                                                    (1 if x == "NONE" else 0)))

stance_labels_test = np.array(twitter_test['Stance'].apply(lambda x: 
                                                                    2 if x == "FAVOR" else 
                                                                    (1 if x == "NONE" else 0)))

In [52]:
#PREP LABELS FOR NN
train_y = np.zeros(shape = (stance_labels_train.shape[0],3))
train_y[stance_labels_train == 0,0] = 1
train_y[stance_labels_train == 1,1] = 1
train_y[stance_labels_train == 2,2] = 1

test_y = np.zeros(shape = (stance_labels_test.shape[0],3))
test_y[stance_labels_test == 0,0] = 1
test_y[stance_labels_test == 1,1] = 1
test_y[stance_labels_test == 2,2] = 1

## Helper function for model performances

In [9]:
def confusion_plot(confusion_matrix, target_names):
    # Plot confusion matrix (via imshow)
    plt.imshow(confusion_matrix, interpolation = "nearest", cmap = plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names)
    plt.yticks(tick_marks, target_names)
    plt.tight_layout()

    # Loop through each value of the matrix to add data labels
    width, height = confusion_matrix.shape
    for x in range(width):
        for y in range(height):
            plt.annotate(str(confusion_matrix[x][y]), xy = (y, x), 
                        horizontalalignment = "center",
                        verticalalignment = "center")
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

In [40]:
def lstm_model(activation_function='relu', 
                optimizer='adam', 
                kernel_initializer=tf.keras.initializers.he_normal(seed=1), 
                bias_initializer='zeros'):
    
    # Define Input layer(s)
    input_vectors = tf.keras.layers.Input(shape=(32,1024), name="input_numbers")
    
    #define LSTM layter
    lstm1 = tf.keras.layers.LSTM(10, 
                                 input_shape = (32,1024),
                                 activation=activation_function,
                                 recurrent_activation=activation_function,
                                 kernel_initializer=kernel_initializer, 
                                 bias_initializer=bias_initializer,
                                 name='lstm')
    #run lstm layer
    lstm_output = lstm1(input_vectors)
    
    #define classification layer (softmax since we have 3 classes)
    classification_layer = tf.keras.layers.Dense(3, activation='softmax', name='classification') # layer definition
    classification_output = classification_layer(lstm_output)   # layer acting on previous layer's output

    # Build and compile model
    lstm_mod = tf.keras.models.Model(input_vectors, classification_output)
    
    lstm_mod.compile(loss='binary_crossentropy', optimizer=optimizer, metrics = ['acc'])
    
    return lstm_mod

lstm_mod = lstm_model()
lstm_mod.summary()
    
    

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_numbers (InputLayer)   [(None, 32, 1024)]        0         
_________________________________________________________________
lstm (LSTM)                  (None, 10)                41400     
_________________________________________________________________
classification (Dense)       (None, 3)                 33        
Total params: 41,433
Trainable params: 41,433
Non-trainable params: 0
_________________________________________________________________


In [41]:
train_vectors['elmo'].shape

(2914, 32, 1024)

In [53]:
#default_train = train_vectors['default'].reshape(2914, 1, 1024)
#default_test = test_vectors['default'].reshape(1956,1,1024)
lstm_mod.fit(
        train_vectors['elmo'],
        train_y,
        epochs=39,
        verbose=1)

W1030 22:05:00.822947 140736218923904 deprecation.py:323] From /Users/alexdessouky/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/39
2914/2914 [==============================] - 3s 907us/sample - loss: nan - acc: 0.6667
Epoch 2/39
2914/2914 [==============================] - 2s 657us/sample - loss: nan - acc: 0.6667
Epoch 3/39
2914/2914 [==============================] - 2s 649us/sample - loss: nan - acc: 0.6667
Epoch 4/39
2914/2914 [==============================] - 2s 638us/sample - loss: nan - acc: 0.6667
Epoch 5/39
2914/2914 [==============================] - 2s 634us/sample - loss: nan - acc: 0.6667
Epoch 6/39
2914/2914 [==============================] - 2s 651us/sample - loss: nan - acc: 0.6667
Epoch 7/39
2914/2914 [==============================] - 2s 651us/sample - loss: nan - acc: 0.6667
Epoch 8/39
2914/2914 [==============================] - 2s 637us/sample - loss: nan - acc: 0.6667
Epoch 9/39
2914/2914 [==============================] - 2s 677us/sample - loss: nan - acc: 0.6667
Epoch 10/39
2914/2914 [==============================] - 2s 673us/sample - loss: nan - acc: 0.6667
Epoch 11/39
2914/29

KeyboardInterrupt: 